In [6]:
from ultralytics import YOLO
model = YOLO('yolov8n-pose.pt')
results = model(source="me.jpg", show=True, conf=0.1, save=True)


WARNING  Environment does not support cv2.imshow() or PIL Image.show()
OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:1301: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'


image 1/1 D:\college\2024_spring\AI\project2\dogs\me.jpg: 448x640 1 person, 224.4ms
Speed: 4.2ms preprocess, 224.4ms inference, 0.0ms postprocess per image at shape (1, 3, 448, 640)
Results saved to runs\pose\predict8


获取视频的总帧数

In [7]:
import cv2  
  
cap = cv2.VideoCapture("badminton.mp4")  
  
if cap.isOpened():  
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))  
    print("Total frames in the video:", total_frames)  
else:  
    print("Error opening video file")  
  
cap.release()

Total frames in the video: 251


下面进行视频逐帧分析，注释掉的部分是用于生成视频片段的，存在bug。后改为生成照片集。

In [35]:
import cv2  
import os  
model = YOLO("yolov8n-pose.pt")  

#判断是否是高远球姿势
def condition_func(frame):   
    # 进行姿态分析  
    results = model(frame)  
    keypoints = results[0].keypoints.xy[0]

    print("----------------")
    #print(keypoints[0][1])
    if keypoints.size(0) == 0:
        return False
    k = 0

    #两个手的点位分别高于两个手臂的点位
    print(keypoints[10], keypoints[8])
    print(keypoints[9], keypoints[7])
    if keypoints[10][1] < keypoints[8][1] and keypoints[9][1] < keypoints[7][1] :
        print("1")
        k += 1 

    #两手距离大于肩宽，手臂张开
    print(keypoints[6], keypoints[5])
    print(keypoints[10], keypoints[9])
    if abs(keypoints[6][0]-keypoints[5][0]) < abs(keypoints[9][0] - keypoints[10][0]):
        print("2")
        k += 1
    if k==2: return True
    else:
        return False
    
  
def process_video(video_path, output_folder, condition_func):  
    if not os.path.exists(output_folder):  
        os.makedirs(output_folder)  
  
    cap = cv2.VideoCapture(video_path)  
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))  
    frame_idx = 0  
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # 输出视频编码格式  
    out = None  # 初始化视频写入对象  
    start_idx = -1  # 初始化满足条件的帧序列的开始索引  
    while cap.isOpened():  
        ret, frame = cap.read()  
        if not ret:  
            break  
  
        if condition_func(frame):  
            # 保存当前帧为图片  
            frame_filename = os.path.join(output_folder, f"frame_{frame_idx:06d}.png")  
            cv2.imwrite(frame_filename, frame)  
            print(f"Saved frame {frame_idx} as {frame_filename}")  
        #     if start_idx == -1:  # 如果这是序列的开始，则创建新的视频写入对象  
        #         start_idx = frame_idx  
        #         out_filename = os.path.join(output_folder, f"clip_{frame_idx//1000:04d}.mp4")  # 使用帧索引（或时间戳）来命名文件  
        #         out = cv2.VideoWriter(out_filename, fourcc, cap.get(cv2.CAP_PROP_FPS), (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))  
  
        #     # 写入当前帧到视频  
        #     out.write(frame)  
        # else:  
        #     if start_idx != -1:  # 如果之前有一个满足条件的序列，现在结束了  
        #         out.release()  # 释放当前视频写入对象  
        #         out = None  
        #         start_idx = -1  # 重置开始索引  
  
        frame_idx += 1  
  
    # 如果在处理结束时有一个未完成的序列，确保它被保存  
    if out is not None:  
        out.release()  
    cap.release()  
  
# 调用函数处理视频  
video_path = "badminton.mp4"  # 替换为你的视频文件路径  
output_folder = "output_clips"  # 输出文件夹  
process_video(video_path, output_folder, condition_func)


0: 384x640 1 person, 250.3ms
Speed: 5.9ms preprocess, 250.3ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)
-------
tensor([647.4560, 440.0916]) tensor([654.5205, 445.1878])
tensor([594.1923, 433.7973]) tensor([608.0629, 439.9031])
1
tensor([648.9238, 427.3699]) tensor([626.0655, 426.2244])
tensor([647.4560, 440.0916]) tensor([594.1923, 433.7973])
2

0: 384x640 1 person, 216.1ms
Speed: 3.0ms preprocess, 216.1ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
-------
tensor([655.9489, 458.5134]) tensor([656.4591, 453.1742])
tensor([587.5693, 434.0476]) tensor([602.6402, 438.7481])
tensor([648.2850, 431.5073]) tensor([621.5196, 428.3117])
tensor([655.9489, 458.5134]) tensor([587.5693, 434.0476])
2

0: 384x640 1 person, 223.4ms
Speed: 2.0ms preprocess, 223.4ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)
-------
tensor([653.3350, 464.8552]) tensor([655.5404, 454.8878])
tensor([597.3972, 446.7994]) tensor([606.9050, 445.5544])
tensor

In [32]:
from ultralytics import YOLO
model = YOLO('yolov8n-pose.pt')
results = model(source="D:\college\\2024_spring\AI基础\project2\寒假打羽毛球的视频.mp4", show=True, conf=0.3, save=True)


WARNING  Environment does not support cv2.imshow() or PIL Image.show()
OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:1301: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'



WARNING  inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/1473) D:\college\